# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Iterative-Methods-for-Solving-Linear-Equations" data-toc-modified-id="Iterative-Methods-for-Solving-Linear-Equations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Iterative Methods for Solving Linear Equations</a></div><div class="lev2 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev2 toc-item"><a href="#PageRank-problem" data-toc-modified-id="PageRank-problem-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>PageRank problem</a></div><div class="lev2 toc-item"><a href="#Jacobi-method" data-toc-modified-id="Jacobi-method-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Jacobi method</a></div><div class="lev2 toc-item"><a href="#Gauss-Seidel-method" data-toc-modified-id="Gauss-Seidel-method-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Gauss-Seidel method</a></div><div class="lev2 toc-item"><a href="#Successive-over-relaxation-(SOR)" data-toc-modified-id="Successive-over-relaxation-(SOR)-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Successive over-relaxation (SOR)</a></div><div class="lev2 toc-item"><a href="#Conjugate-gradient-method" data-toc-modified-id="Conjugate-gradient-method-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Conjugate gradient method</a></div><div class="lev2 toc-item"><a href="#MatrixDepot.jl" data-toc-modified-id="MatrixDepot.jl-17"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>MatrixDepot.jl</a></div><div class="lev2 toc-item"><a href="#Numerical-examples" data-toc-modified-id="Numerical-examples-18"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Numerical examples</a></div><div class="lev3 toc-item"><a href="#Generate-test-matrix" data-toc-modified-id="Generate-test-matrix-181"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>Generate test matrix</a></div><div class="lev3 toc-item"><a href="#Matrix-vector-muliplication" data-toc-modified-id="Matrix-vector-muliplication-182"><span class="toc-item-num">1.8.2&nbsp;&nbsp;</span>Matrix-vector muliplication</a></div><div class="lev3 toc-item"><a href="#Dense-solve-via-Cholesky" data-toc-modified-id="Dense-solve-via-Cholesky-183"><span class="toc-item-num">1.8.3&nbsp;&nbsp;</span>Dense solve via Cholesky</a></div><div class="lev3 toc-item"><a href="#Jacobi-solver" data-toc-modified-id="Jacobi-solver-184"><span class="toc-item-num">1.8.4&nbsp;&nbsp;</span>Jacobi solver</a></div><div class="lev3 toc-item"><a href="#Gauss-Seidal" data-toc-modified-id="Gauss-Seidal-185"><span class="toc-item-num">1.8.5&nbsp;&nbsp;</span>Gauss-Seidal</a></div><div class="lev3 toc-item"><a href="#SOR" data-toc-modified-id="SOR-186"><span class="toc-item-num">1.8.6&nbsp;&nbsp;</span>SOR</a></div><div class="lev3 toc-item"><a href="#Conjugate-Gradient" data-toc-modified-id="Conjugate-Gradient-187"><span class="toc-item-num">1.8.7&nbsp;&nbsp;</span>Conjugate Gradient</a></div>

# Iterative Methods for Solving Linear Equations

In [ ]:
versioninfo()

## Introduction

So far we have considered direct methods for solving linear equations.    

* **Direct methods** (flops fixed _a priori_) vs **iterative methods**:
    - Direct method (GE/LU, Cholesky, QR, SVD): good for dense, small to moderate sized $\mathbf{A}$  
    - Iterative methods (Jacobi, Gauss-Seidal, SOR, conjugate-gradient, GMRES): good for large, sparse, structured linear system, parallel computing, warm start


## PageRank problem

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/400px-PageRanks-Example.svg.png" width="300" align="center"/>

* $\mathbf{A}  \in \{0,1\}^{n \times n}$ the connectivity matrix of webpages with entries
$$
\begin{eqnarray*}
	a_{ij} = \begin{cases}
	1 &  \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}. 
\end{eqnarray*}
$$
$n \approx 10^9$ in May 2017.

* $r_i = \sum_j a_{ij}$ is the *out-degree* of page $i$. 

* [Larry Page](https://en.wikipedia.org/wiki/PageRank) imagines a random surfer wandering on internet according to following rules:
    - From a page $i$ with $r_i>0$
        * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
        * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page 
    - From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page
    
The process defines a Markov chain on the space of $n$ pages. Stationary distribution of this Markov chain gives the ranks (probabilities) of each page.

* Stationary distribution is the top left eigenvector of the transition matrix $\mathbf{P}$ corresponding to eigenvalue 1. Equivalently it can be cast as a linear equation
$$
    (\mathbf{I} - \mathbf{P}^T) \mathbf{x} = \mathbf{0}.
$$

* Gene Golub: Largest matrix computation in world. 

* GE/LU will take $2 \times (10^9)^3/3/10^{12} \approx 6.66 \times 10^{14}$ seconds $\approx 20$ million years on a tera-flop supercomputer!

* Iterative methods come to the rescue.

## Jacobi method

<img src="https://www.usna.edu/Users/math/meh/Jacobi.jpeg" width="150" align="center"/>

Solve $\mathbf{A} \mathbf{x} = \mathbf{b}$.

* Jacobi iteration: 
$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$

* With splitting: $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$, Jacobi iteration can be written as
$$
    \mathbf{D} \mathbf{x}^{(t+1)} = - (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{b},
$$
i.e., 
$$
	\mathbf{x}^{(t+1)} = - \mathbf{D}^{-1} (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b} = - \mathbf{D}^{-1} \mathbf{A} \mathbf{x}^{(t)} + \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b}.
$$

* One round costs $2n^2$ flops with an **unstructured** $\mathbf{A}$. Gain over GE/LU if converges in $o(n)$ iterations. Saving is huge for **sparse** or **structured** $\mathbf{A}$. By structured, we mean the matrix-vector multiplication $\mathbf{A} \mathbf{v}$ is fast ($O(n)$ or $O(n \log n)$).

## Gauss-Seidel method

<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/Carl_Friedrich_Gauss.jpg" width="150" align="center"/>

<img src="http://www.scientificlib.com/en/Physics/Biographies/images/Thumbs/ThLudwigVonSeidel01.jpg" width="150" align="center"/>

* Gauss-Seidel (GS) iteration:
$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$

* With splitting, $(\mathbf{D} + \mathbf{L}) \mathbf{x}^{(t+1)} = - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b}$, i.e., 
$$
\mathbf{x}^{(t+1)} = - (\mathbf{D} + \mathbf{L})^{-1} \mathbf{U} \mathbf{x}^{(t)} + (\mathbf{D} + \mathbf{L})^{-1} \mathbf{b}.
$$

* GS converges for any $\mathbf{x}^{(0)}$ for symmetric and pd $\mathbf{A}$.

* Convergence rate of Gauss-Seidel is the spectral radius of the $(\mathbf{D} + \mathbf{L})^{-1} \mathbf{U}$.

## Successive over-relaxation (SOR)

* SOR: 
$$
x_i^{(t+1)} = \omega \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)} \right) / a_{ii} + (1-\omega) x_i^{(t)},
$$
i.e., 
$$
\mathbf{x}^{(t+1)} = (\mathbf{D} + \omega \mathbf{L})^{-1} [(1-\omega) \mathbf{D} - \omega \mathbf{U}] \mathbf{x}^{(t)} + (\mathbf{D} + \omega \mathbf{L})^{-1} (\mathbf{D} + \mathbf{L})^{-1} \omega \mathbf{b}.
$$

* Need to pick $\omega \in [0,1]$ beforehand, with the goal of improving convergence rate.

## Conjugate gradient method

* **Conjugate gradient and its variants are the top-notch iterative methods for solving huge, structured linear systems.**

* A UCLA invention! Hestenes and Stiefel in 60s.

* Solving $\mathbf{A} \mathbf{x} = \mathbf{b}$ is equivalent to minimizing the quadratic function $\frac{1}{2} \mathbf{x}^T \mathbf{A} \mathbf{x} - \mathbf{b}^T \mathbf{x}$. 

[Kershaw's results](http://www.sciencedirect.com/science/article/pii/0021999178900980?via%3Dihub) for a fusion problem.

| Method                                 | Number of Iterations |
|----------------------------------------|----------------------|
| Gauss Seidel                           | 208,000              |
| Block SOR methods                      | 765                  |
| Incomplete Cholesky conjugate gradient | 25                   |

## MatrixDepot.jl

MatrixDepot.jl is an extensive collection of test matrices in Julia. After installation, we can check available test matrices by

In [ ]:
using MatrixDepot

mdinfo()

In [ ]:
# List matrices that are positive definite and sparse:
mdlist(:symmetric & :posdef & :sparse)

In [ ]:
# Get help on Poisson matrix
mdinfo("poisson")

In [ ]:
# Generate a Poisson matrix of dimension n=10
A = matrixdepot("poisson", 10)

In [ ]:
using UnicodePlots
spy(A)

In [ ]:
# Get help on Wathen matrix
mdinfo("wathen")

In [ ]:
# Generate a Wathen matrix of dimension n=5
A = matrixdepot("wathen", 5)

In [ ]:
spy(A)

## Numerical examples

The [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package implements most commonly used iterative solvers.

### Generate test matrix

In [ ]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 100
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)

In [ ]:
spy(A)

In [ ]:
# storage difference
Base.summarysize(A), Base.summarysize(Afull)

### Matrix-vector muliplication

In [ ]:
# randomly generated vector of length n^2
b = randn(n^2)
# dense matrix-vector multiplication
@benchmark $Afull * $b

In [ ]:
# sparse matrix-vector multiplication
@benchmark $A * $b

### Dense solve via Cholesky

In [ ]:
# record the Cholesky solution
xchol = cholesky(Afull) \ b;

In [ ]:
# dense solve via Cholesky
@benchmark cholesky($Afull) \ $b

### Jacobi solver

It seems that Jacobi solver doesn't give the correct answer.

In [ ]:
xjacobi = jacobi(A, b)
@show norm(xjacobi - xchol)

Reading [documentation](https://juliamath.github.io/IterativeSolvers.jl/dev/linear_systems/stationary/#Jacobi-1) we found that the default value of `maxiter` is 10. A couple trial runs shows that 30000 Jacobi iterations are enough to get an accurate solution.

In [ ]:
xjacobi = jacobi(A, b, maxiter=30000)
@show norm(xjacobi - xchol)

In [ ]:
@benchmark jacobi($A, $b, maxiter=30000)

### Gauss-Seidal

In [ ]:
# Gauss-Seidel solution is fairly close to Cholesky solution after 15000 iters
xgs = gauss_seidel(A, b, maxiter=15000)
@show norm(xgs - xchol)

In [ ]:
@benchmark gauss_seidel($A, $b, maxiter=15000)

### SOR

In [ ]:
# symmetric SOR with ω=0.75
xsor = ssor(A, b, 0.75, maxiter=10000)
@show norm(xsor - xchol)

In [ ]:
@benchmark sor($A, $b, 0.75, maxiter=10000)

### Conjugate Gradient

In [ ]:
# conjugate gradient
xcg = cg(A, b)
@show norm(xcg - xchol)

In [ ]:
@benchmark cg($A, $b)